In [1]:
using Optim

In [2]:
harmonic(n::Integer) = quadgk(x->((1-x^n)/(1-x)), zero(n), one(n)) |> first

harmonic (generic function with 1 method)

In [3]:
function harmonic(n::Integer, m)
    if m==one(m)
        harmonic(n)
    else
        ret = zero(float(m))
        for k in 1:n #TODO: Good to do this in Parallel for n>10^6
            ret += inv(k^m)
        end
        ret
    end
end

harmonic (generic function with 2 methods)

In [4]:
harmonic(30,2.5)

1.3375300748835552

In [5]:
using Distributions
import Distributions: DiscreteUnivariateDistribution, pdf, cdf, logpdf, logcdf, suffstats, fit_mle
import Base: rand, minimum, maximum, mean
import StatsBase.mode
using StatsBase

In [6]:
immutable Zepf{N<:Integer, T1<:Real,T2<:Real} <: DiscreteUnivariateDistribution
    h::T1
    n::N
    s::T2
end

function Zepf{N,T2}(n::N, s::T2)
    n>zero(s) || error("d must be greater than 0 (d=$d)")
    s>one(s) || error("s must be greater than 1 (s=$s)")
    Zepf(harmonic(n,s),n, s)
end

Zepf{N<:Integer,T1<:Real,T2<:Real}

In [7]:
function rand{N,T1}(d::Zepf{N,T1})
    x=Base.rand()
    r=one(N)
    while(0<(x-=pdf(d,r)))
        r+=one(r)
    end
    if r>d.n 
        #Handles floats not quiet lining up, allocate excess probility to first  and thus largest category
        one(N)
    else
        r
    end
end

pdf(d::Zepf, k::Int) = inv(d.h * k^d.s)
logpdf(d::Zepf, k::Int) = -log(d.h) - d.s*log(k)
cdf(d::Zepf, k::Int) = harmonic(k,d.s)/d.h
logcdf(d::Zepf, k::Int) = log(harmonic(k,d.s)) - log(d.h)

#quantile is annoying to implement; but there is probably a really smart way to do it, since we do know the distribution
minimum{N}(::Zepf{N})=one(Int)#(N)
maximum{N}(d::Zepf{N})=d.n |> Int

mean{N,T1,T2}(d::Zepf{N,T1,T2})=harmonic(d.n,d.s-one(T2))/d.h
mode{N}(d::Zepf{N}) = one(N)

mode (generic function with 55 methods)

In [8]:
cdf(Zepf(big"1000",big"1.999"), 1)


6.079525638595357693974084878656164556746957827721866348098622511730935776786549e-01

In [9]:
immutable ZepfStats{N<:Integer} <: SufficientStats
    freqs::Vector{N} #Counts of how many times each element occurs, sorted by rank
    n::N
end

ZepfStats{N}(freqs::Vector{N}) = ZepfStats(freqs, length(freqs))


function suffstats{T<:Integer}(::Type{ZepfStats}, x::AbstractArray{T})
    freqs = counts(x)
    issorted(counts, rev=true) || warn("Catagory IDs are not proportional to frequency. To fit Zeph well, one expects the category freqency to be monotonically decreasing with catagory id.")
    ZepfStats(freqs)
end

suffstats (generic function with 34 methods)

In [99]:
function fit_mle(ss::ZepfStats)
    #Switch to http://stats.stackexchange.com/questions/6780/how-to-calculate-zipfs-law-coefficient-from-a-set-of-top-frequencies
    function f(x)
        s=x[1]
        h=harmonic(ss.n, s)
        comp(k)=ss.freqs[k] - (h*k)^-s
        sumabs2(comp.(1:ss.n))
    end
    
    
    best_result=optimize(f, [1.0], BFGS())
    for initial in (2.0).^(0:0.005:100)
        result = optimize(f, [initial], BFGS(),
        OptimizationOptions(g_tol=eps(0.0)))
        s = result.minimum[1]
        if s>1.0 && (s<best_result.minimum[1] || best_result.minimum[1]<1.0)
            best_result=result
        end
    end
    best_result
end

fit_mle (generic function with 53 methods)

In [102]:
result=fit_mle( ZepfStats([8000,4000,2000,1000,500]))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [32.334446287576434]
 * Minimizer: [32.33634888958691]
 * Minimum: 8.523400e+07
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 4.9e-324: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 6
 * Gradient Calls: 6

In [100]:
result=fit_mle( ZepfStats([26486, 12053, 5052, 3033, 2536, 2391, 1444, 1220, 1152, 1039]))

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [14.928527864588919]
 * Minimizer: [30.325022783032964]
 * Minimum: 8.995802e+08
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 4.9e-324: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 4
 * Gradient Calls: 4

1-element Array{Float64,1}:
 -1.20458

In [12]:
using Optim
foo(x) =  x[1]^2-x[1]-2
result = optimize(foo, [1.0], BFGS())

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [1.0]
 * Minimizer: [0.4999999999973902]
 * Minimum: -2.250000e+00
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 4
 * Gradient Calls: 4